In [1]:
import numpy as np  
import pandas as pd 

# Предварительная обработка

In [3]:
# подгружаем датасет
df = pd.read_csv(path + '/parsed_cars.csv')
df = df.drop(['Unnamed: 0', 'index', 'Unnamed: 0.1'], axis = 1) # убираем ненужные столбцы

df.head()

,Название,Ссылка,Цена,Год,Километраж,Город,Класс автомобиля,Количество дверей,Количество мест,Длина,Ширина,Высота,Коробка передач,Тип привода,Максимальная скорость,Разгон до 100,Тип двигателя,Максимальная мощность
0,BYD E2,https://auto.ru/cars/used/sale/byd/e2/11231094...,2450000₽,2023.0,1.0,moskva,C,5.0,5,4260.0,1760.0,1530.0,Автомат,передний,130.0,NaN,Электро,95 л.с. (70 кВт)
1,Changan UNI-K,https://auto.ru/cars/used/sale/changan/uni_k/1...,3440000₽,2023.0,1.0,moskva,E,5.0,5,4865.0,1948.0,1695.0,Автомат,полный,200.0,NaN,бензиновый,226 л.с. (166 кВт) при 5500
2,LiXiang L9 44.5 kWh,https://auto.ru/cars/used/sale/lixiang/l9/1122...,7760000₽,2024.0,1.0,moskva,F,5.0,6,5218.0,1998.0,1800.0,Автомат,полный,180.0,5.3,гибридный,449 л.с. (330 кВт)
3,NaN,https://auto.ru/cars/used/sale/geely/monjaro/1...,NaN,NaN,NaN,moskva,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Zeekr 007 770,https://auto.ru/cars/used/sale/zeekr/007/11229...,5830000₽,2024.0,1.0,moskva,D,4.0,5,4865.0,1900.0,1450.0,Автомат,полный,210.0,3.5,Электро,646 л.с. (475 кВт)


In [4]:
df.columns # наши полученные переменные

Index(['Название', 'Ссылка', 'Цена', 'Год', 'Километраж', 'Город',
       'Класс автомобиля', 'Количество дверей', 'Количество мест', 'Длина',
       'Ширина', 'Высота', 'Коробка передач', 'Тип привода',
       'Максимальная скорость', 'Разгон до 100', 'Тип двигателя',
       'Максимальная мощность'],
      dtype='object')

### Описание переменных
1. **Название**: Название марки автомобиля и ее модели
2. **Ссылка**: Ссылка на объявления на сайте auto.ru
3. **Цена**: Целевая переменная, которую мы потом будем предсказывать. Содержит информацию о цене машины в рублях
4. **Год**: Описывает год выпуска автомобиля
5. **Километраж**: Расстояние, которое машина проехала в км
6. **Город**: Город, в котором было размещено объявление. Были собраны объявления из 6 городов: Москва, Ростов-на-Дону, Санкт-Петербург, Екатеринбург, Новосибирск, Владивосток
7. **Класс автомобиля**: Всего представлено 8 классов машин, информация о классах есть [тут](https://www.autonews.ru/news/627e0cad9a79476cc13b0bef)
8. **Количество дверей**: Показывает, сколько дверей в машине. В нашем случае либо 4, либо 5
9. **Количество мест**: Показывает сколько максимально людей машина может вместить
10. **Длина, ширина, высота**: Содержит информацию о размерах машины в миллиметрах
11. **Коробка передач**: Тип коробки передач машины. Всего есть 4 типа: автомат, робот, вариатор, механика
12. **Тип привода**: Информация о приводе автомобиля (передний, полный или задний)
13. **Максимальная скорость**: Максимальная скорость, до которой может разогнаться автомобиль в км/ч
14. **Разгон до 100**: Время в секундах, за которое машина разгоняется до 100 км/ч
15. **Тип двигателя**: Показывает тип двигателя автомобиля: бензиновый, дизельный, электро или гибридный
16. **Максимальная мощность**: Максимальная мощность двигателя машины в л.с.

В данных есть несколько проблем: надо убрать пустые строки, достать значения цен и мощности двигателя в формате int, а также убрать наблюдения с неизвестной ценой (их всего 2)

In [5]:
# убираем пустые строки
df = df[df.isna().sum(axis = 1) < 10] 

#убираем строки с неизвестной ценой, так как это целевая переменная
df = df[df['Цена'].isna() == False] 

#достаем значение макс. мощности из строки
df['Максимальная мощность'].apply(lambda w: int(w.split('л.с.')[0].strip())) 

#убираем для цены символ рубля и делаем тип int
df['Цена'] = df['Цена'].str.replace('₽', '').astype(float) 

Посмотрим на значения в переменной "Количество мест". 

In [6]:
df['Количество мест'].value_counts()

5          2343
7           178
5, 7         86
6            75
4, 5         62
4            32
4, 7         22
6, 7         20
5, 6, 7      15
4, 6, 7      11
6, 4         10
6, 7, 8       2
5, 7, 9       1
2             1
Name: Количество мест, dtype: int64

Видим, что для некоторых моделей машин указывается несколько вариантов числа мест. Мы будем брать максимальную вместимость:

In [7]:
df['Количество мест'] = df['Количество мест'].apply(lambda x: x.split(', ')[-1])

Теперь посмотрим на пропуски в данных. Пропуски есть только в двух переменных: максимальная скорость и разгон до 100, мы постараемся их заполнить. Так как для автомобилей внутри одного класса эти показатели довольно близкие, мы заполним их средней по классу.

In [8]:
df.isna().sum()

Название                   0
Ссылка                     0
Цена                       0
Год                        0
Километраж                 0
Город                      0
Класс автомобиля           0
Количество дверей          0
Количество мест            0
Длина                      0
Ширина                     0
Высота                     0
Коробка передач            0
Тип привода                0
Максимальная скорость    187
Разгон до 100            675
Тип двигателя              0
Максимальная мощность      0
dtype: int64

In [9]:
df[df['Максимальная скорость'].isna()]['Класс автомобиля'].value_counts()

C    85
E    25
J    24
D    19
M    17
B    12
F     5
Name: Класс автомобиля, dtype: int64

In [10]:
for i in np.where(df['Максимальная скорость'].isna())[0]:
    df['Максимальная скорость'].iloc[i] = df[df['Класс автомобиля'] == 
                                    df['Класс автомобиля'].iloc[i]]['Максимальная скорость'].mean()
    
for i in np.where(df['Разгон до 100'].isna())[0]:
    df['Разгон до 100'].iloc[i] = df[df['Класс автомобиля'] == 
                                    df['Класс автомобиля'].iloc[i]]['Разгон до 100'].mean()

/var/folders/80/wx2xrtm12cn4rjgnh957mx1m0000gn/T/ipykernel_80215/3299269934.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Максимальная скорость'].iloc[i] = df[df['Класс автомобиля'] ==
/var/folders/80/wx2xrtm12cn4rjgnh957mx1m0000gn/T/ipykernel_80215/3299269934.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Разгон до 100'].iloc[i] = df[df['Класс автомобиля'] ==


In [11]:
df.isna().sum()

Название                 0
Ссылка                   0
Цена                     0
Год                      0
Километраж               0
Город                    0
Класс автомобиля         0
Количество дверей        0
Количество мест          0
Длина                    0
Ширина                   0
Высота                   0
Коробка передач          0
Тип привода              0
Максимальная скорость    0
Разгон до 100            0
Тип двигателя            0
Максимальная мощность    0
dtype: int64